# Importing all necesarry modules and setting up pandas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
%matplotlib inline
pd.set_option('display.max_columns',None)
dataset=pd.read_csv("data/train.csv")

## Catagoric Feature Case Handeling

In [2]:
catagorical_nan_feature=[feature for feature in dataset.columns if dataset[feature].dtype=='O' 
                         and dataset[feature].isnull().sum()>1]
def fill_na_cat(data,feature):
    data=dataset.copy()
    data[feature]=data[feature].fillna("missing")
    return data
dataset=fill_na_cat(dataset,catagorical_nan_feature)

## Numeric feature case handeling

In [3]:
numeric_nan_feature=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 
                     and dataset[feature].dtypes!= 'O']
for feature in numeric_nan_feature:
    median=dataset[feature].median()
    dataset[feature+'na']=np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(median,inplace=True)
# dataset

# Date time feature handeling

In [4]:
temporal_feature=[feature for feature in dataset.columns if 'Yr' in feature or 'Year' in feature]
for feature in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    dataset[feature]=dataset['YrSold']-dataset[feature]

## Handeling rare catagorical feature

In [5]:
catagorical_feature=[feature for feature in dataset.columns if dataset[feature].dtype =='O']
for feature in catagorical_feature:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.1].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_na')
dataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagena,MasVnrAreana,GarageYrBltna
0,1,60,RL,65.0,8450,Pave,missing,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,5,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,missing,Attchd,5.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,missing,missing,missing,0,2,2008,WD,Normal,208500,0,0,0
1,2,20,RL,80.0,9600,Pave,missing,Reg,Lvl,AllPub,Rare_na,Gtl,Rare_na,Rare_na,Norm,1Fam,1Story,6,8,31,31,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Rare_na,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,31.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,missing,missing,missing,0,5,2007,WD,Normal,181500,0,0,0
2,3,60,RL,68.0,11250,Pave,missing,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,7,6,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Rare_na,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,7.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,missing,missing,missing,0,9,2008,WD,Normal,223500,0,0,0
3,4,70,RL,60.0,9550,Pave,missing,IR1,Lvl,AllPub,Corner,Gtl,Rare_na,Norm,Norm,1Fam,2Story,7,5,91,36,Gable,CompShg,Wd Sdng,Rare_na,None,0.0,TA,TA,Rare_na,TA,Rare_na,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,8.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,missing,missing,missing,0,2,2006,WD,Rare_na,140000,0,0,0
4,5,60,RL,84.0,14260,Pave,missing,IR1,Lvl,AllPub,Rare_na,Gtl,Rare_na,Norm,Norm,1Fam,2Story,8,5,8,8,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,8.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,missing,missing,missing,0,12,2008,WD,Normal,250000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,missing,Reg,Lvl,AllPub,Inside,Gtl,Rare_na,Norm,Norm,1Fam,2Story,6,5,8,7,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,8.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,missing,missing,missing,0,8,2007,WD,Normal,175000,0,0,0
1456,1457,20,RL,85.0,13175,Pave,missing,Reg,Lvl,AllPub,Inside,Gtl,Rare_na,Norm,Norm,1Fam,1Story,6,6,32,22,Gable,CompShg,Rare_na,Rare_na,Rare_na,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rare_na,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Rare_na,2,TA,Attchd,32.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,missing,MnPrv,missing,0,2,2010,WD,Normal,210000,0,0,0
1457,1458,70,RL,66.0,9042,Pave,missing,Reg,Lvl,AllPub,Inside,Gtl,Rare_na,Norm,Norm,1Fam,2Story,7,9,69,4,Gable,CompShg,Rare_na,Rare_na,None,0.0,Rare_na,Rare_na,Rare_na,TA,Rare_na,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,69.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,missing,Rare_na,Rare_na,2500,5

# Here took those continious feature who have no zero values(Because log(0)=infinity)

In [6]:
for feature in ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea','SalePrice']:
    dataset[feature]=np.log(dataset[feature])

In [7]:
def encoding(data,feature_list):
    df=dataset.copy()
    for feature in feature_list: 
        mean_encoding=df.groupby([feature])['SalePrice'].mean().to_dict()
        df[feature]=df[feature].map(mean_encoding)
    return df
df=encoding(dataset,catagorical_feature)

In [8]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagena,MasVnrAreana,GarageYrBltna
0,1,60,12.085885,4.174387,9.041922,12.025522,12.037675,11.936094,12.022694,12.024183,12.001900,12.020565,12.163641,12.042916,12.025895,12.047512,12.182686,7,5,5,5,11.98420,12.019216,12.205873,12.209631,12.163630,196.0,12.311282,12.043078,12.261665,12.179882,12.042615,11.955653,12.301937,706,12.043738,0,150,856,12.032078,12.207626,12.06109,12.061464,6.752270,854,0,7.444249,1,0,2,1,3,1,12.222337,8,12.038511,0,11.808907,12.160778,5.0,12.172672,2,548,12.068539,12.070483,12.059895,0,61,0,0,0,0,12.022115,12.058599,12.030112,0,2,2008,11.991061,12.005034,12.247694,0,0,0
1,2,20,12.085885,4.382027,9.169518,12.025522,12.037675,11.936094,12.022694,12.024183,12.178605,12.020565,12.037104,11.905200,12.025895,12.047512,11.993683,6,8,31,31,11.98420,12.019216,11.859767,11.862286,11.892919,0.0,11.837985,12.043078,11.870080,12.179882,12.042615,12.148250,11.953313,978,12.043738,0,284,1262,12.032078,12.207626,12.06109,12.061464,7.140453,0,0,7.140453,0,1,2,0,3,1,11.810592,6,12.038511,1,12.186416,12.160778,31.0,12.172672,2,460,12.068539,12.070483,12.059895,298,0,0,0,0,0,12.022115,12.058599,12.030112,0,5,2007,11.991061,12.005034,12.109011,0,0,0
2,3,60,12.085885,4.219508,9.328123,12.025522,12.037675,12.163465,12.022694,12.024183,12.001900,12.020565,12.163641,12.042916,12.025895,12.047512,12.182686,7,5,7,6,11.98420,12.019216,12.205873,12.209631,12.163630,162.0,12.311282,12.043078,12.261665,12.179882,12.042615,12.148250,12.301937,486,12.043738,0,434,920,12.032078,12.207626,12.06109,12.061464,6.824374,866,0,7.487734,1,0,2,1,3,1,12.222337,6,12.038511,1,12.186416,12.160778,7.0,12.172672,2,608,12.068539,12.070483,12.059895,0,42,0,0,0,0,12.022115,12.058599,12.030112,0,9,2008,11.991061,12.005034,12.317167,0,0,0
3,4,70,12.085885,4.094345,9.164296,12.025522,12.037675,12.163465,12.022694,12.024183,12.027446,12.020565,12.037104,12.042916,12.025895,12.047512,12.182686,7,5,91,36,11.98420,12.019216,11.838506,11.987987,11.892919,0.0,11.837985,12.043078,11.710539,11.810855,11.860713,11.955653,11.953313,216,12.043738,0,540,756,12.032078,11.910371,12.06109,12.061464,6.867974,756,0,7.448334,1,0,1,0,3,1,12.222337,7,12.038511,1,12.256533,11.765643,8.0,11.818974,3,642,12.068539,12.070483,12.059895,0,35,272,0,0,0,12.022115,12.058599,12.030112,0,2,2006,11.991061,12.111007,11.849398,0,0,0
4,5,60,12.085885,4.430817,9.565214,12.025522,12.037675,12.163465,12.022694,12.024183,12.178605,12.020565,12.037104,12.042916,12.025895,12.047512,12.182686,8,5,8,8,11.98420,12.019216,12.205873,12.209631,12.163630,350.0,12.311282,12.043078,12.261665,12.179882,12.042615,12.158267,12.301937,655,12.043738,0,490,1145,12.032078,12.207626,12.06109,12.061464,7.043160,1053,0,7.695303,1,0,2,1,4,1,12.222337,9,12.038511,1,12.186416,12.160778,8.0,12.172672,3,836,12.068539,12.070483,12.059895,192,84,0,0,0,0,12.022115,12.058599,12.030112,0,12,2008,11.991061,12.005034,12.429216,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Here we are going to do feature scaling

In [9]:
scaling_feature=[feature for feature in df.columns if 'Id' not in feature and 'SalePrice' not in feature]
scaler=MinMaxScaler()
scaling_data=scaler.fit_transform(df[scaling_feature])
data=pd.DataFrame(data=scaling_data,columns=scaling_feature)
final_data=pd.concat([dataset[['Id','SalePrice']],data],axis=1)

# And finally here is our cook ready to feature selection and apply for machine learning algorithim

In [11]:
df.to_csv('x_train.csv',index=False)